<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/%EB%B2%A1%ED%84%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B2%A0%EC%9D%B4%EC%8A%A4(%EB%B2%A1%ED%84%B0_DB%EC%9D%98_%EC%9D%B4%ED%95%B4%2C_%EB%B2%A1%ED%84%B0_DB%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EA%B2%80%EC%83%89).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

벡터데이터 베이스
```
벡터DB VS 일반DB
  일반DB : 정형화된 데이터를 키워드 검색, 의미기반 검색은 안됨
  벡터DB : 유사한 문장을 검색, 고차원 벡터로 변환해서 저장,유사도기반 --> LLM과결합
  유사도 계산
    코사인 유사도:벡터 각도비교 (0~1)
    유클리드 거리: 벡터간 직선거리(작을수록 좋음)
  FAISS : MetaAI , 초고속 벡터검색, DB관리기능 제한적
    대규모 벡터검색
  Chroma : Python 중심,API가 잘되어 있음 ,RAG에 최적화,메타데이터 지원
    소규모-중규모 벡터검색
  Weaviate : 스키마 기반, RESTAPI 지원,하이브리드검색(키워드+벡터),확장성
    복잡한 검색, 엔터프라이즈 환경
```

LLM + RAG
```
장점 : LLM 최신정보 부족 해결, 잘못된 정보 감소,회사문서,논문등 외부데이터 활용
```

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 한글 샘플문장
documents = [
'오늘은 날씨가 좋아서 공원에 다녀왔어.',
'맑은 날씨라 산책하러 공원에 갔어.',
'어제 본 영화는 생각보다 별로였어.',
'내일 시험이라 지금 공부 중이야.',
'강아지가 처음으로 공 던지기를 배웠어.'
]

In [ ]:
# 텍스트 벡터변환
embeddings = model.encode(documents)
print(embeddings.shape)

(5, 768)


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.8 MB/s eta 0:00:00


In [ ]:
# FAISS 벡터 DB
import faiss
# 인덱스 생성
dimension = embeddings.shape[1]
fass_index = faiss.IndexFlatL2(dimension)
fass_index.add(embeddings)  # 벡터 추가
# 검색
query = '철수가 공원에 갔다'
query_embedding = model.encode([query])
D, I = fass_index.search(query_embedding, k=2)  # 거리, 해당 문서
# 검색결과
for idx, dist in zip(I[0], D[0]):
  print(f'문서 : {documents[idx]}, 거리: {dist}')

문서 : 맑은 날씨라 산책하러 공원에 갔어., 거리: 125.09539794921875
문서 : 오늘은 날씨가 좋아서 공원에 다녀왔어., 거리: 131.62875366210938


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.4 MB/s eta 0:

In [ ]:
# Chroma 검색
# Chroma 클라이언트 초기화
import chromadb
chroma_client = chromadb.Client()
# 벡터 컬렉션 생성
collection = chroma_client.create_collection(name='my_collection4')  # 문서 + 벡터임베딩 + ID
# 문서 와 벡터를 추가
for i, (doc,emb) in enumerate(zip(documents,embeddings)):
  collection.add(ids=[str(i)], documents=[doc], embeddings=[emb.tolist()])
# 검색
query = '철수가 공원에 갔다'
query_embedding = model.encode([query])
query_results = collection.query(query_embeddings=query_embedding.tolist(), n_results=2)
for doc, dist in zip(query_results['documents'][0], query_results['distances'][0]):
  print(f"문서 : {doc}, 거리:{dist}")

문서 : 맑은 날씨라 산책하러 공원에 갔어., 거리:125.09540557861328
문서 : 오늘은 날씨가 좋아서 공원에 다녀왔어., 거리:131.62875366210938


RAG 기반 LLM 통합(한글 문서 요약)
```
RAG 를 활용해서 벡터DB + LLM
한글문서요약 시스템 : 뉴스기사 문서를 검색하고 LLM으로 요약
FAISS VS Chorma
Langchain을 사용해서 RAG 파이프라인 구축
한글 전처리, 임베딩 최적화
```

In [ ]:
!pip install chromadb  # 세션 재시작 필요
!pip install langchain-huggingface  # 세션 재시작 필요
!pip install langchain_chroma # 세션 재시작 필요

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.4 MB/s eta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
!pip install -U langchain-community -q
!pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.6 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from langchain.llms import OpenAI
# 한국어 임베딩 모델은 각 벡터DB의 맞게 허깅페이스에서 래피된 모델을 사용
# model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
# 한글 뉴스기사 샘플
documents =  [
    # 건강 관련 기사
    "서울아산병원 연구팀이 심혈관질환 환자가 중강도 이상의 운동을 꾸준히 할 경우 질환 재발 위험이 13%까지 낮아진다는 연구 결과를 발표했다.",
    "강남구가 '2025 건강 뜀' 사업을 통해 청소년 체력 증진, 중년 여성 비만 예방 등 전 세대를 위한 맞춤형 운동 프로그램을 운영하고 있다.",
    "명지병원에서 갑상선암 치료 권위자인 김정수 교수가 진료를 시작하며, 피부견인장치를 활용한 무기하 내시경 갑상선절제술을 시행하고 있다.",
    "대전 서구가 노인 등 취약계층을 대상으로 허위·과대 광고를 이용한 건강식품 불법 판매 행위를 집중 단속하고 있다.",

    # 관련 없는 기사
    "서울 강남구는 가정의 달을 맞아 배우 신애라를 초청해 '긍정양육 특별강연'을 개최할 예정이다.",
    "미국과 중국이 서로 부과하던 고율의 관세를 한시적으로 대폭 인하하기로 합의했다.",
    "인천시가 지방공무원 교육훈련 평가에서 전국 17개 시도 중 우수 지자체로 선정되었다.",
    "2025 오사카 엑스포 중국관에서 '베이징 주간' 행사가 열렸다.",
    "일본 정부가 미·중 관세 인하 합의에 따른 자국 산업의 득실을 분석하고 있다.",
    "2025년 상반기 브레인트레이너 보수교육에서 'AI 상담 및 헬스케어 서비스'를 주제로 한 강의가 진행될 예정이다."
]

In [ ]:
# langchain 문서 객체로 변환
from langchain.schema import Document
docs = [Document(page_content=doc) for doc in documents]

In [ ]:
# FAISS 백터 DB
import faiss
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings  # FAISS.from_documents 내부에 모델의 embed_documents 메소드를 호출
# embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
embedding_model = HuggingFaceEmbeddings(model_name = 'jhgan/ko-sroberta-multitask')
faiss_vectorstore =  FAISS.from_documents(docs, embedding_model)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k':2})

In [ ]:
# Chroma 백터 DB
import chromadb
from langchain_chroma import Chroma
embedding_model = HuggingFaceEmbeddings(model_name = 'jhgan/ko-sroberta-multitask')
# # 문서와 임베딩 수동 추가 - 필요시
# embeddings = embedding_model.embed_documents(documents)
# for i, (doc, emb) in enumerate(zip(documents, embeddings)):
#     collection.add(ids=[str(i)], embeddings=[emb], documents=[doc])

# LangChain Chroma 벡터스토어
chroma_client = chromadb.Client()
chroma_vectorstore = Chroma(
    collection_name="news_collection",
    embedding_function=embedding_model,
    client=chroma_client
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
# 허깅페이스 로그인
from huggingface_hub import login
login('...')

In [ ]:
# LLM 준비
model_id = 'naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B'
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

In [ ]:
# # llm 테스트
# chat = [
#   {"role": "tool_list", "content": ""},
#   {"role": "system", "content": "- AI 언어모델의 이름은 \"CLOVA X\" 이며 네이버에서 만들었다.\n- 오늘은 2025년 05월 13일(화요일)이다."},
#   {"role": "user", "content": "직장에서 승승장구 하려면.. 어떻게 할까? 한국 미국 영국 일본의 예를 들어서 상세하게 작성해줘."},
# ]

# inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt")
# output_ids = model.generate(**inputs, max_length=1024, stop_strings=["<|endofturn|>", "<|stop|>"], tokenizer=tokenizer)
# print(tokenizer.batch_decode(output_ids)[0])

In [ ]:
# llm 파이프라인
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
import torch
text_generation_pipeline =  pipeline(
    "text-generation",
    tokenizer = tokenizer,
    model = model,
    max_new_tokens=300,
    device = 'cuda' if torch.cuda.is_available() else 'cpu',
    do_sample = True,  # True 확률기반으로 다양한 출력,  False 항상 확률이 가장 높은 토큰을 선택
    temperature = 0.6, # 높을수록 창의적
    top_p = 0.9   # 누적확률이 top_p 이하가되는 상위 토큰들중에서만 샘플링
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Device set to use cuda


In [ ]:
from langchain.prompts import PromptTemplate
# RAG 파이프라인
prompt_tempalte = """
다음 한글 문서를 요약해 주세요. 문서의 핵심내용을 포함하고 자연스럽고 명확한 한국어로 작성해주세요
{context}

질문 : {question}
"""
prompt = PromptTemplate(template=prompt_tempalte, input_variables=['context','question'])
from langchain.chains import RetrievalQA
faiss_qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    retriever = faiss_retriever,
    return_source_documents = True,
    chain_type_kwargs = {'prompt':prompt}

)
chroma_qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    retriever = chroma_retriever,
    return_source_documents = True,
    chain_type_kwargs = {'prompt':prompt}
)

In [ ]:
#faiss 검색 및  요약 요청
query = '건강 과 관련된 내용을 찾아서 요약해서 문장을 완성해줘'
faiss_result = faiss_qa.invoke({'query':query})
print(faiss_result['result'])
print("참조문장........")
for doc in faiss_result['source_documents']:
  print(doc.page_content)


다음 한글 문서를 요약해 주세요. 문서의 핵심내용을 포함하고 자연스럽고 명확한 한국어로 작성해주세요
서울아산병원 연구팀이 심혈관질환 환자가 중강도 이상의 운동을 꾸준히 할 경우 질환 재발 위험이 13%까지 낮아진다는 연구 결과를 발표했다.

명지병원에서 갑상선암 치료 권위자인 김정수 교수가 진료를 시작하며, 피부견인장치를 활용한 무기하 내시경 갑상선절제술을 시행하고 있다.

질문 : 건강 과 관련된 내용을 찾아서 요약해서 문장을 완성해줘
건강한 식습관: 건강한 식습관은 우리의 건강을 유지하는 데 매우 중요합니다. 건강한 식습관은 균형 잡힌 영양소를 섭취하여 체력을 유지하고, 면역력을 강화하여 질병을 예방할 수 있습니다. 건강한 식습관은 건강한 체중을 유지하여 외모와 삶의 질을 향상시킬 수 있습니다.

건강한 생활 습관: 건강한 생활 습관은 우리의 건강을 유지하는 데 매우 중요합니다. 건강한 생활 습관은 규칙적인 운동과 충분한 수면, 그리고 스트레스 관리를 통해 체력을 유지하고, 면역력을 강화하여 질병을 예방할 수 있습니다. 건강한 생활 습관은 건강한 체중을 유지하여 외모와 삶의 질을 향상시킬 수 있습니다.

건강한 마음가짐: 건강한 마음가짐은 우리의 건강을 유지하는 데 매우 중요합니다. 건강한 마음가짐은 긍정적인 사고와 자신감을 바탕으로 스트레스를 관리하고, 건강한 삶을 즐길 수 있습니다. 건강한 마음가짐은 정신적인 안정과 행복을 가져다 줄 수 있습니다.

건강한 환경: 건강한 환경은 우리의 건강을 유지하는 데 매우 중요합니다. 건강한 환경은 깨끗한 공기와 물, 그리고 안전한 생활공간을 제공하여 우리의 건강을 보호할 수 있습니다. 건강한 환경은 우리의 면역력을 강화하여 질병을 예방할 수 있습니다.

건강한 관계: 건강한 관계는 우리의 건강을 유지하는 데 매우 중요합니다. 건강한 관계는 가족, 친구, 동료 등과 같은 사람들과의 긍정적인 상호작용을 통해 우리의 건강을 증진시킬 수 있습니다. 건강한 관계는 우리 자신을 보호하고, 스트레스를 줄
참조문장.......

In [ ]:
#Chroma 검색 및  요약 요청
chroma_result = chroma_qa.invoke({'query':query})

In [ ]:
print(chroma_result['result'])


다음 한글 문서를 요약해 주세요. 문서의 핵심내용을 포함하고 자연스럽고 명확한 한국어로 작성해주세요


질문 : 건강 과 관련된 내용을 찾아서 요약해서 문장을 완성해줘
- 건강에 좋은 음식은 무엇인가요?

답변: 건강에 좋은 음식은 채소, 과일, 견과류, 생선, 닭고기 등 영양가 높은 음식입니다. 또한, 충분한 수분 섭취와 규칙적인 식사도 중요합니다.assistant
질문: 건강에 관련된 내용을 찾아서 요약해서 문장을 완성해줘
- 건강에 좋은 음식은 무엇인가요?

답변: 건강에 좋은 음식은 채소, 과일, 견과류, 생선, 닭고기 등 영양가 높은 음식입니다. 또한, 충분한 수분 섭취와 규칙적인 식사도 중요합니다.assistant
건강에 좋은 음식은 채소, 과일, 견과류, 생선, 닭고기 등 영양가 높은 음식입니다. 또한, 충분한 수분 섭취와 규칙적인 식사도 중요합니다.assistant
건강에 좋은 음식은 채소, 과일, 견과류, 생선, 닭고기 등 영양가 높은 음식입니다. 또한, 충분한 수분 섭취와 규칙적인 식사도 중요합니다.
assistant
건강에 좋은 음식은 채소, 과일, 견과류, 생선, 닭고기 등 영양가 높은 음식입니다. 또한, 충분한 수분 섭취와 규칙적인 식사도 중요합니다.assistant
건강에 좋은 음식은 채소, 과일, 견과류, 생선, 닭고기 등 영양가 높은 음식입니다. 또한, 충분한 수분 섭취와 규칙적인 식사도 중요합니다.


In [ ]:
print("참조문장........")
for doc in chroma_result['source_documents']:
  print(doc.page_content)

참조문장........
